# 🚨 Katastrophen-Tweets: NLP Master-Projekt
### Automatisierte Analyse, Modellierung und Benchmarking

Dieses Notebook demonstriert den Einsatz der zentralen Bibliothek `nlp_utils.py`, um eine vollständige NLP-Pipeline aufzubauen.
Es umfasst:
1.  **Daten laden & EDA**: Verteilungen, Wordclouds, N-Gramme.
2.  **Modell-Benchmark**: Ein Wettkampf zwischen 25+ Algorithmen (Speed vs. Quality).
3.  **Deep Learning**: Training eines BERT-Modells (Transformer).
4.  **Ensemble-Optimierung**: Auto-Tuning und Hard Example Mining.

---

In [1]:
# 1. SETUP & IMPORTS
import sys
import os
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split

# Warnungen unterdrücken für saubere Ausgabe
warnings.filterwarnings("ignore")

# Füge den 'Muster'-Ordner zum Pfad hinzu, um nlp_utils zu importieren
sys.path.append(os.path.abspath("Muster"))

# Importiere ALLES aus unserer mächtigen Bibliothek
from nlp_utils import *

print(f"✅ Setup abgeschlossen. Datenpfad: {TRAIN_DATA_PATH}")

✅ Setup abgeschlossen. Datenpfad: /Users/cristallagus/Desktop/GitHub/Verarbeitung_nat-rlicher_Sprache_Katastrophen-Tweets/Data_Set/train.csv


In [2]:
# KONFIGURATION
# Setze None für den vollen Datensatz oder eine Zahl (z.B. 500) für schnelles Testen
MAX_SAMPLES = None #daten begrenzer

# 2. DATEN LADEN & SPLIT
try:
    # 1. Daten laden
    df_full = pd.read_csv(TRAIN_DATA_PATH)

    # 2. Flexible Reduzierung (Falls MAX_SAMPLES gesetzt ist)
    if MAX_SAMPLES is not None and len(df_full) > MAX_SAMPLES:
        # Wir nutzen sample mit stratify (indirekt über random_state),
        # um die Klassenverteilung beim Kürzen zu erhalten
        df_full = df_full.sample(n=MAX_SAMPLES, random_state=42).reset_index(drop=True)
        print(f"⚠️ Stichprobe aktiviert: Nutze nur {MAX_SAMPLES} Reihen (Entwicklungs-Modus).")
    else:
        print(f"✅ Voller Datensatz geladen: {df_full.shape}")

    # 3. Vorbereitung der Spalten
    if 'cleaned_text' not in df_full.columns:
        df_full['cleaned_text'] = ""

    # 4. Stratifizierter Split (80/20)
    # y muss neu definiert werden, falls gesampelt wurde
    df_train, df_val = train_test_split(
        df_full,
        test_size=0.2,
        random_state=42,
        stratify=df_full['target']
    )

    del df_full
    gc.collect()

    print(f"Trainings-Set: {df_train.shape}")
    print(f"Validierungs-Set: {df_val.shape}")

    # 5. df_Cleaning initialisieren (für die nachfolgende Pipeline)
    df_Cleaning = df_train.copy()

    # Deine Regel: Für jede Visualisierung/Zusammenfassung eine Tabelle zeigen
    print("\n📊 Numerische Übersicht des gewählten Trainings-Sets:")
    display(df_Cleaning['target'].value_counts().to_frame())
    display(df_Cleaning.head())

except FileNotFoundError:
    print(f"🚨 FEHLER: Datei unter {TRAIN_DATA_PATH} nicht gefunden.")
except Exception as e:
    print(f"🚨 Ein unerwarteter Fehler ist aufgetreten: {e}")

✅ Voller Datensatz geladen: (7613, 2)
Trainings-Set: (6090, 3)
Validierungs-Set: (1523, 3)

📊 Numerische Übersicht des gewählten Trainings-Sets:


,count
target,
0,3473
1,2617


,text,target,cleaned_text
6234,Sassy city girl country hunk stranded in Smoky...,1,
326,God's Kingdom (Heavenly Gov't) will rule over ...,0,
997,Mopheme and Bigstar Johnson are a problem in t...,0,
7269,@VixMeldrew sounds like a whirlwind life!,0,
2189,Malaysia confirms plane debris washed up on Re...,1,


# EDA

In [3]:
# Globale Übersicht (Ist-Zustand vor der Reinigung)
EDA_vor_reinigung(df_train)

Mancos in der Struktur (NaNs, Duplikate, Leerräume)
Mancos in der Sprache (Fremdsprachen, reine Emoji-Texte, zu kurze Texte)
Mancos im Inhalt (Slang/Kürzel, Stoppwort-Last, Vokabular-Dichte)
Mancos in der Verteilung (Wie unterscheiden sich Katastrophen von normalen Tweets?)
                🚀 TEXT-DATEN-ANALYSE (IST-ZUSTAND)

🌍 SPRACHANALYSE (Stichprobe 500 Einträge):


,Anzahl (Count),Anteil (%)
text,,
en,476,95.2
da,5,1.0
de,4,0.8
no,3,0.6
ca,2,0.4
et,2,0.4
nl,2,0.4
sq,1,0.2
too_short,1,0.2



📊 ÜBERSICHT DER SPALTEN-QUALITÄT:


,Spalte,NaN (Mancos),Unique,Kardinalität (%),Ø Zeichen,Texte m. Emojis,Texte m. Kürzeln,URLs,Sonderzeichen
0,text,0,6018,98.82,101.1,575,1294,3179,34158
1,target,0,2,0.03,-,-,-,-,-
2,cleaned_text,0,1,0.02,0.0,0,0,0,0



🧠 STRUKTURELLE TEXT-ANALYSE (KOMPLEXITÄT & NOISE):


...

,Ø Länge,Ø Großbuchst.,Texte m. Emojis
target,,,
0,95.50,9.24,288
1,108.47,10.92,287



📏 STATISTIK DER ROH-TEXTLÄNGEN:


,count,mean,std,min,25%,50%,75%,max
Zeichen-Anzahl,6090.0,101.071264,33.741949,7.0,77.0,107.0,133.0,157.0
Wort-Anzahl,6090.0,14.897865,5.699638,1.0,11.0,15.0,19.0,31.0



⚠️ GEFUNDENE ANOMALIEN / MANCOS:


,Anomalie-Typ,Anzahl (Count),Bedeutung
0,Leere Texte / Nur Whitespace,0,Kein Info-Gehalt
1,Extrem kurz (< 5 Zeichen),0,Kaum Kontext
2,Reine Emoji-Texte (kein Alphabet),0,Nicht sprachlich auswertbar
3,Gekürzte Begriffe (Shortforms),1291,Informelle Sprache
4,Shouting (Nur Großbuchstaben),42,Extreme Emotionalität
5,Text-Duplikate (Spam-Gefahr),72,Verzerrt Vokabular
6,Inkonsistente Umbrüche (\n),340,Struktur-Rauschen


# BEREINIGUNG

In [4]:
# Standard Bereinigung (Erstellt df_Cleaning)
# Dies führt Übersetzung, Emoji-Umwandlung und Lemmatisierung durch
df_Cleaning = standard_Cleare_TEXT(df_train, ziel_sprache='en')

🚀 START NLP-PIPELINE: DF_CLEANING
DEBUG: Spalten vor Reinigung: ['text', 'target', 'cleaned_text']
🧠 Verarbeite 6090 Zeilen parallel (Core-Reserve aktiv)...


NLP Pipeline:   0%|                    | 0/6090 [00:00<?, ?it/s]


📊 ZUSAMMENFASSUNG DER REINIGUNG:
            Metrik  Wert
     Zeilen Vorher  6090
    Zeilen Nachher  5544
Duplikate entfernt   546
✅ REINIGUNG BEENDET: df_Cleaning ist bereit für Radar-Plot Signale.


# Erweiterte EDA

In [5]:
# 3. EXPLORATIVE DATENANALYSE (EDA)
# Wir nutzen nun das bereinigte DataFrame df_Cleaning für die Analyse

# 3.2 Visualisierungen
plot_target_distribution(df_Cleaning)


📊 NUMERISCHE VERTEILUNG:


,label,count,Anteil (%)
0,Keine Katastrophe,3274,59.05
1,Katastrophe,2270,40.95


### 📊 Interpretation: Verteilung der Zielvariable
Dieses Balkendiagramm zeigt das Verhältnis zwischen echten Katastrophen-Tweets (Rot) und normalen Tweets (Grün).
- Ein **ausgewogenes Verhältnis** ist ideal für das Training.
- Wenn eine Klasse stark dominiert (z.B. 90% Nicht-Katastrophen), muss das Modell mit Techniken wie `class_weight='balanced'` oder Resampling gegensteuern, um nicht nur die Mehrheitsklasse vorherzusagen.

In [6]:
# 3. Textlängen-Analyse (Histogramm & Violin)
# Vergleich der Zeichenanzahl zwischen Target 0 und 1
print('Verteilung der Textlängen (0 vs 1)')
plot_metric_by_target(df_Cleaning, metric_col='length')

Verteilung der Textlängen (0 vs 1)



📊 STATISTIK: length


,Status,count,mean,std,min,25%,50%,75%,max
0,Katastrophe,2270.0,108.101762,29.835523,17.0,87.0,115.0,136.0,151.0
1,Keine Katastrophe,3274.0,94.739462,35.981174,7.0,66.0,99.0,129.0,157.0


In [7]:
# 1. Signal-Stärke der Keywords (Top 20)
# Dies zeigt, welche Wörter am stärksten auf eine Katastrophe hindeuten
df_TEMP_KW = df_Cleaning.copy()
df_TEMP_KW['found_keyword'] = df_TEMP_KW['cleaned_text'].apply(get_hit_keyword)
print(f"📊 Analyse der Signal-Keywords")
print(f"Anzahl der analysierten Tweets: {len(df_TEMP_KW)}")
plot_feature_signal_ratio(df_TEMP_KW[df_TEMP_KW['found_keyword'].notna()],
                          feature_col='found_keyword',
                          top_n=40)
del df_TEMP_KW

📊 Analyse der Signal-Keywords
Anzahl der analysierten Tweets: 5544



📊 DATENTABELLE: found_keyword


,found_keyword,mean,count
0,outbreak,1.000000,16
1,derailment,1.000000,18
2,hiroshima,0.976744,43
3,killed,0.863636,22
4,earthquake,0.840000,25
5,tornado,0.793103,29
6,missing,0.789474,19
7,murder,0.750000,52
8,drought,0.736842,19
9,terror,0.733333,45


In [8]:
# 2. Geografische Signale (Top 15 Locations)
df_TEMP_LOC = df_Cleaning.copy()
df_TEMP_LOC['found_location'] = df_TEMP_LOC['cleaned_text'].apply(get_hit_location)
print('Signal-Stärke der Locations')
print(f"🌍 Analyse der geografischen Signale (aus Text)")
plot_feature_signal_ratio(df_TEMP_LOC[df_TEMP_LOC['found_location'].notna()],feature_col='found_location',top_n=15)
del df_TEMP_LOC

Signal-Stärke der Locations
🌍 Analyse der geografischen Signale (aus Text)



📊 DATENTABELLE: found_location


,found_location,mean,count
0,forest,0.804878,41
1,bridge,0.800000,25
2,street,0.724138,29
3,building,0.697674,86
4,station,0.680000,25
5,house,0.588235,51
6,park,0.550000,20
7,downtown,0.545455,11
8,hospital,0.500000,10
9,mall,0.500000,14


### 📊 Interpretation: Tweet-Länge
Dieses Histogramm vergleicht die Länge der Tweets (Anzahl der Zeichen) für beide Klassen.
- Überlappen sich die Kurven stark, ist die Länge kein gutes Unterscheidungsmerkmal.
- Oft sind Katastrophen-Tweets (Rot) etwas länger und detaillierter als alltägliche Kurznachrichten.
- Spitzen bei bestimmten Längen können auf automatisierte Bots oder Retweets hindeuten.

In [9]:
# 4. Phrasen-Analyse (Bigramme / 2-Wort-Kombinationen)
# Findet die häufigsten Wortpaare im gereinigten Text
print('Häufigste Bigramme (Wortpaare)')
plot_neutral_ngrams(df_Cleaning, n=2, top_n=15)

Häufigste Bigramme (Wortpaare)



📊 TOP 15 2-GRAMM DATEN:


,Count,N-Gram
0,41,gon na
1,41,burning building
2,37,look like
3,31,suicide bomber
4,30,mhthree seventy
5,28,body bag
6,27,wan na
7,26,thousand dollar
8,26,liked video
9,25,mass murder


### 📊 Interpretation: N-Gramm Analyse (Bigramme)
Dieses Balkendiagramm zeigt die häufigsten Wortpaare (Bigramme).
- Einzelne Wörter können mehrdeutig sein ("fire" -> Kaminfeuer oder Waldbrand?).
- Bigramme liefern Kontext: "forest fire" vs. "fire place".
- Diese Analyse hilft zu verstehen, welche Phrasen starke Signale für das Modell sind.

In [10]:
plot_wordclouds(df_Cleaning)


📊 TOP 10 WÖRTER PRO KATEGORIE:


,Katastrophe,Normal
0,dollar (1787),dollar (2281)
1,the (980),the (1462)
2,and (817),a (1061)
3,in (788),i (1016)
4,a (734),and (1006)
5,of (645),to (903)
6,to (550),of (679)
7,hundred (409),in (619)
8,on (310),is (582)
9,is (310),you (555)


# FEATURE ENGINEERING

# Aktueller Stopp words

In [11]:
# Wenn stop_words_combined Set oder eine Liste ist:
print("Auszug der kombinierten Stopwords:")
stop_words_combined = set(stopwords.words('english'))
print(list(stop_words_combined)[:])

# Falls du wissen willst, wie viele Wörter insgesamt gefiltert werden:
print(f"\nGesamtanzahl der Stopwords: {len(stop_words_combined)}")

Auszug der kombinierten Stopwords:
['with', 'hadn', 'doing', 'couldn', "should've", "mustn't", 'about', 'its', 'nor', "aren't", "shouldn't", "you've", 'same', 'again', 'needn', 'this', 'mustn', 'were', 'weren', 'd', 'herself', 'had', 'do', 'before', "haven't", "i've", "she'll", 'they', 'is', 'are', "it'd", "mightn't", 'm', 'mightn', 'than', 'up', 'out', 'll', "you're", "needn't", 'don', 'was', 'ain', 'all', 'an', 'have', 'there', 'theirs', "couldn't", 'for', 'has', "didn't", 'most', 'or', 'themselves', "wasn't", 'in', "she's", 'over', 'too', 'be', 'while', 'shan', 'because', 'when', 'ma', "it's", 'myself', "we've", 'if', 'of', 'other', 'our', 'very', 'we', "doesn't", 'more', 'from', 'so', 'ourselves', 'yourself', 'his', 'didn', 'just', 'on', 'during', 'that', 'against', 'will', 'wouldn', 'which', "we'd", "he's", 'no', "won't", 'me', 'hasn', 'any', "we'll", 'she', "they'll", "it'll", "we're", "shan't", "hasn't", 'it', 'haven', 'these', 'down', 'those', 'yours', 'until', 'am', 'not', 'i'

# Erkannte mögliche Stopwords und Marker für Katastrophen Erkannt

In [12]:
# 5. Strategische Token-Analyse (Signal vs. Rauschen)
# Vergleicht die häufigsten Wörter in beiden Klassen und identifiziert potenzielle Stoppwörter
print('Strategische Token-Analyse (Signal vs. Rauschen)')
plot_strategic_token_analysis(df_Cleaning, top_n=30, height=600)

Strategische Token-Analyse (Signal vs. Rauschen)



🔍 STRATEGISCHE AUSWERTUNG FÜR MODELL-OPTIMIERUNG


,Typ,Tokens
0,STOPPWORT-VORSCHLAG,"with, and, it, for, hundred, am, a, like, not,..."
1,ECHTE SIGNALE,"after, at, by, fire, from, have, two, wa"



SIND MANUELL IM NÄCHSTEN CODE EINZUFÜGEN


# Erstelle eigene stopwords Liste

In [13]:
# stop worts manuelle Anpassung
print(get_strategic_stopwords())


🛡️ SIGNAL-ENGINEERING KONFIGURATION:
| Kategorie                    |   Anzahl |
|:-----------------------------|---------:|
| Aktive Filter (Noise)        |      190 |
| Gerettete Signale (Features) |       10 |

💡 Beispiele geretteter Signale: ['after', 'are', 'as', 'at', 'by']...
{'with', 'hadn', 'doing', 'couldn', "should've", "mustn't", 'about', 'its', 'nor', "aren't", "shouldn't", "you've", 'like', 'same', 'again', 'needn', 'this', 'mustn', 'were', 'weren', 'd', 'herself', 'had', 'do', 'before', "haven't", "i've", "she'll", 'they', 'is', "it'd", "mightn't", 'm', 'mightn', 'than', 'up', 'out', 'll', "you're", "needn't", 'don', 'was', 'ain', 'all', 'an', 'have', 'there', 'theirs', "couldn't", 'for', 'has', "didn't", 'most', 'or', 'themselves', "wasn't", 'in', "she's", 'over', 'too', 'be', 'shan', 'because', 'when', 'ma', "it's", 'myself', "we've", 'if', 'of', 'other', 'our', 'very', 'we', "doesn't", 'more', 'so', 'ourselves', 'yourself', 'his', 'didn', 'just', 'on', 'that', 'again

# Erstelle mehr spalten vergleich, schwächere modelle ungenauigkeiten zu stärken der erkennung

In [14]:
#erstelle Engeneering mehrere neue spalten
df_Cleaning = run_full_disaster_pipeline(df_Cleaning)


🚀 STARTE ADVANCED FEATURE ENGINEERING (OS: Darwin)
🚀 Berechne Metriken (Raw-Quelle: text, Clean-Quelle: cleaned_text)...
🔍 Schritt 1: Kontext-Extraktion...
📝 Schritt 2: Punctuation & Caps Metrics...
🔗 Schritt 5: Source Credibility...
🎯 Schritt 6: Erzeuge Triaden-Signale (Type, Location, Time)...
📈 Schritt 7: Berechne Noise-Flag... 
📈 Schritt 8: Berechne finalen Disaster-Score...
🤖 Schritt 9: Enclave LLM Analyse (Core-Reserve aktiv)...

📦pyarrow installation überprüfung
✅(pyarrow) ist bereits einsatzbereit.

🛠️ AUTOMATISCHES ENCLAVE-SETUP (Darwin erkannt)...
🖥️ Hardware-Status: GPU/MPS BESCHLEUNIGT
🎯 Ziel-Modell: llama3
✅ Modell llama3 ist bereits einsatzbereit.
🚀 Starte LLM-Engineering (Ollama: llama3) für 5544 Tweets...


Enclave (llama3):   0%|                    | 0/5544 [00:00<?, ?it/s]

💾 Schablone sicher in Ablagerung_NLP/enclave abgelegt.

📊 ENCLAVE STATUS-BERICHT (Numerische Unterstützung):


,count
enclave_score,
1,4312
0,1232



📊 ENGINEERING ABGESCHLOSSEN. Numerische Übersicht für Radar-Plot:
       type_K  type_Location  type_time  type_K_L_t_combi  is_triple_signal  \
mean  0.44228       0.121627   0.367244          0.006494               0.0   
min  -1.00000       0.000000   0.000000          0.000000               0.0   
max   1.00000       1.000000   1.000000          1.000000               0.0   

      disaster_score  is_noise_flag  
mean        1.278319       0.162879  
min         0.000000       0.000000  
max       268.000000       1.000000  

📊 Generiere Polar-Plot basierend auf den neuen Features...



📊 SORTIERTE BASISDATEN (Mittelwerte & 0-10 Analyse):


,Ø Keine Katastrophe,Ø Katastrophe,Delta_Score_0_10
has_url,0.3882,0.6326,2.4439
type_K,0.3082,0.6357,1.6375
enclave_score,0.7126,0.8718,1.5922
length,94.7395,108.1018,0.8908
is_noise_flag,0.1912,0.1220,0.6918
is_news_style,0.3879,0.4471,0.5923
type_Location,0.0976,0.1563,0.5871
subjectivity,0.3232,0.2676,0.5566
emoji_count,8.9282,20.2357,0.5026
word_count,14.6823,15.2952,0.2043



📋 Vorschau der ersten 5 Zeilen (df_Cleaning):


# ML Auswahl und endscheidung
Für den datensatz welche modelle und rechen geschwindigkeiten jedes modell Hatt das in frage kommt

# Erster Anlauf standard evaluierung mit Kleinen modell 'spalten wahl möglich im code'

# 1 = Katastrofe

In [15]:
# MASTER-STUDIE & ENSEMBLE (INTEGRIERT)

# 1. DATEN-SYNCHRONISATION
print("Bereite Master-Daten aus dem Engineering-Output (df_Cleaning) vor...")

from sklearn.model_selection import train_test_split

X_train_master, X_test_master, y_train_master, y_test_master = train_test_split(df_Cleaning,
                                                                                df_Cleaning['target'],
                                                                                test_size=0.2,
                                                                                random_state=42,
                                                                                stratify=df_Cleaning['target'])

# 2. STOPWORD-DEFINITION
try:
    current_stop_words = stop_words_combined
except NameError:
    current_stop_words = get_strategic_stopwords()

# 3. MASTER-ANALYSE STARTEN (Inkl. Mining 6.0 & 1 Core Frei Standard)
print(f"\n🚀 Starte Master-Studie mit Mining-Gewicht 6.0...")
model = run_master_study_ensemble(
    X_train_master,
    X_test_master,
    y_train_master,
    y_test_master,
    stop_words=current_stop_words
)

# 4. NUMERISCHE VALIDIERUNG
print("\n📊 Zusammenfassung der Master-Daten:")
validation_data = {
    "Metrik": ["Samples", "Features (NLP)", "Mining Gewicht", "Core Reserve"],
    "Trainings-Set": [len(X_train_master), "length, caps, subjectivity", "6.0", "n_jobs=-2"],
    "Test-Set": [len(X_test_master), "length, caps, subjectivity", "N/A", "Aktiv"]
}
display(pd.DataFrame(validation_data))

# 5. FINALE REINIGUNG
del X_train_master, X_test_master
gc.collect()

Bereite Master-Daten aus dem Engineering-Output (df_Cleaning) vor...

🚀 Starte Master-Studie mit Mining-Gewicht 6.0...
STUDIE TEIL 1: SIGNAL- UND RAUSCH-ANALYSE

📊 TOP 10 SIGNALWÖRTER (Threshold >= 5):


,Wort,Relevanz,Total
254,severe,33.00,33
863,debris,25.00,25
1467,bomber,23.00,23
645,mhthree,23.00,23
21,migrant,22.00,22
1477,suspect,21.00,21
1717,spill,18.00,18
1076,hiroshima,17.50,36
1007,seventy,17.33,54
1652,mosque,17.00,17



SCHRITT 2: FEATURE PREPARATION & LEMMATISIERUNG

🚀 Starte Ensemble Training (Core-Reserve: 7 Cores)...


Ensemble GridSearch:   0%|                    | 0/18 [00:00<?, ?it/s]


🔍 SCHRITT 4: HARD EXAMPLE MINING (Refit mit Gewicht 6.0)...

### STUDIE TEIL 3: FINALE ERGEBNISSE (Interaktiv) ###


,index,precision,recall,f1-score,support
0,0,0.86,0.70,0.77,655.00
1,1,0.66,0.83,0.74,454.00
2,accuracy,0.76,0.76,0.76,0.76
3,macro avg,0.76,0.77,0.75,1109.00
4,weighted avg,0.78,0.76,0.76,1109.00



📊 Zusammenfassung der Master-Daten:


,Metrik,Trainings-Set,Test-Set
0,Samples,4435,1109
1,Features (NLP),"length, caps, subjectivity","length, caps, subjectivity"
2,Mining Gewicht,6.0,N/A
3,Core Reserve,n_jobs=-2,Aktiv


1359

In [16]:
# 4. MODELL-BENCHMARK (TECHNISCHE MODELL-WAHL & RESSOURCEN-CHECK)

# 1. SELEKTION: 5 längste Zeilen für den technischen Speed-Test
print("Selektiere 5 längste Tweets für Ressourcen-Benchmark...")

# Wir sortieren nach Länge und nehmen alle Spalten für die Feature-Pipeline
df_mini_bench = df_Cleaning.sort_values(by=COLS['length'], ascending=False).head(5).copy()

X_train_bench = df_mini_bench.copy()
y_train_bench = df_mini_bench['target']
X_val_bench   = df_mini_bench.copy()
y_val_bench   = df_mini_bench['target']

# 2. BENCHMARK STARTEN (Wahl des effizientesten Modells)
print(f"🚀 Starte Modell-Wahl (Basis: Zeit & Ressourcen auf {len(X_train_bench)} Samples)...")
print("Starte Benchmark...")
benchmark_results = run_comprehensive_benchmark(X_train_bench, y_train_bench, X_val_bench, y_val_bench, current_stop_words)
# 3. FINALE REINIGUNG
print("\n🧹 Bereinige Benchmark-Ressourcen aus dem RAM...")
del df_mini_bench, X_train_bench, X_val_bench, y_train_bench, y_val_bench
gc.collect()

Selektiere 5 längste Tweets für Ressourcen-Benchmark...
🚀 Starte Modell-Wahl (Basis: Zeit & Ressourcen auf 5 Samples)...
Starte Benchmark...
### START: Benchmark ###
### System-Check ###
OS: Darwin 24.6.0
CPU Kerne: 8
RAM: 16.00 GB
✅ GPU gefunden: 1x [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

⏱️ Starte Effizienz-Check (Worst-Case Szenario)...



Starte vollen Benchmark (Warnungen werden unterdrückt)...


Benchmarking:   0%|                    | 0/29 [00:00<?, ?it/s]


⚠️ 2 Warnungen wurden während des Benchmarks unterdrückt.
            Model  Training Time (s)  Prediction Time (s)  Recall  F1-Score  Accuracy  Precision
           LogReg           0.006632             0.002787     1.0       1.0       1.0        1.0
              SGD           0.018953             0.013071     1.0       1.0       1.0        1.0
            Ridge           0.008037             0.002701     1.0       1.0       1.0        1.0
PassiveAggressive           0.005327             0.001815     1.0       1.0       1.0        1.0
       Perceptron           0.017617             0.002199     1.0       1.0       1.0        1.0
          RidgeCV           0.005669             0.001985     1.0       1.0       1.0        1.0
   Multinomial NB           0.004464             0.001808     1.0       1.0       1.0        1.0
     Bernoulli NB           0.004326             0.001878     1.0       1.0       1.0        1.0
    Complement NB           0.004947             0.002563     1.0   


📊 BENCHMARK DATEN (TOP 10 NACH Recall):


,Model,Recall,F1-Score,Accuracy,Training Time (s)
1,LogReg,1.0,1.0,1.0,0.0066
14,Decision Tree,1.0,1.0,1.0,0.0046
27,Gaussian Process,1.0,1.0,1.0,0.0063
26,QDA,1.0,1.0,1.0,0.0043
25,LDA,1.0,1.0,1.0,0.0105
23,Nearest Centroid,1.0,1.0,1.0,0.0039
20,Gradient Boosting,1.0,1.0,1.0,0.0277
19,AdaBoost,1.0,1.0,1.0,0.0051
18,Bagging,1.0,1.0,1.0,0.0207
17,Extra Trees,1.0,1.0,1.0,0.0656



🧹 Bereinige Benchmark-Ressourcen aus dem RAM...


2849

### 📊 Interpretation: Modell-Benchmark (Speed vs. Quality)
Dieser Scatterplot ist entscheidend für die Modellauswahl.
- **X-Achse (Log-Skala)**: Trainingszeit in Sekunden. Je weiter links, desto schneller.
- **Y-Achse**: F1-Score (Qualität). Je höher, desto besser.
- **Der "Sweet Spot"**: Modelle oben links (schnell & gut).
- Oft sind **Lineare Modelle** (Logistic Regression, SGD) extrem schnell und überraschend gut.
- **Transformer (BERT)** wären ganz oben, aber sehr weit rechts (langsam).
- Dieser Plot hilft zu entscheiden: Brauche ich maximale Performance (BERT) oder reicht ein schnelles Modell für Echtzeit-Anwendungen?

In [17]:
# 5. HAUPT-ANALYSE (Strikt nach nlp_utilsV1.py Standards)


# 1. DATEN-VORBEREITUNG
X_train_master = df_Cleaning.copy()
y_train_master = df_Cleaning[COLS['target']]

X_test_master = df_val.copy()
y_test_master = df_val[COLS['target']]

# 2. FEATURE-CHECK (Sicherstellen, dass deine globalen Namen existieren in df_val)
for df_ptr in [X_train_master, X_test_master]:
    if COLS['length'] not in df_ptr.columns:
        df_ptr[COLS['length']] = df_ptr[COLS['text']].str.len()

    if 'is_noise_flag' not in df_ptr.columns:
        df_ptr['is_noise_flag'] = 0
    if 'disaster_score' not in df_ptr.columns:
        df_ptr['disaster_score'] = 0.0

# 2. FEATURE-INTEGRITÄT PRÜFEN
required_cols = [COLS['length'], 'is_noise_flag', 'disaster_score']

# 3. START DER ANALYSE (Nutzt deine Core-Reserve Logik)
print(f"🚀 Starte Haupt-Analyse (Reserved Cores: {RESERVED_CORES})...")
best_ensemble = run_analysis(X_train_master, y_train_master, X_test_master, y_test_master, current_stop_words)

# 5. NUMERISCHE VALIDIERUNG
final_stats = {"Datensatz": ["Training", "Validierung"],
               "Anzahl Samples": [len(X_train_master), len(X_test_master)],
               "Features": [X_train_master.shape[1], X_test_master.shape[1]]}
display(pd.DataFrame(final_stats))

🚀 Starte Haupt-Analyse (Reserved Cores: 7)...
 START: Analyse Pipeline (Recall-Optimiert)
🔍 Prüfe/Generiere Features für Datensatz (5544 Zeilen)...
🔍 Prüfe/Generiere Features für Datensatz (1523 Zeilen)...


GridSearch Ensemble:   0%|                    | 0/12 [00:00<?, ?it/s]

⛏️ Mining: 1 harte Fälle gefunden. Refit...

📊 FINALE METRIKEN:


,precision,recall,f1-score,support
0,0.555019,0.55531,0.54559,1523.0



📊 NUMERISCHE BASISDATEN (Konfusionsmatrix):


,Vorhergesagt: Keine Katastrophe,Vorhergesagt: Katastrophe
Ist: Keine Katastrophe,423,446
Ist: Katastrophe,246,408


,Datensatz,Anzahl Samples,Features
0,Training,5544,22
1,Validierung,1523,6


### 📊 Interpretation: Konfusionsmatrix & ROC-Kurve
**Konfusionsmatrix (Links):**
- Zeigt, wo das Modell Fehler macht.
- **Oben rechts (False Positive)**: Falscher Alarm (Normaler Tweet als Katastrophe erkannt).
- **Unten links (False Negative)**: Verpasste Gefahr (Katastrophe als normal erkannt) -> **Das wollen wir vermeiden!**

**Trennschärfe (Rechts):**
- Zeigt die Wahrscheinlichkeitsverteilung.
- Idealerweise sind die roten Balken (Katastrophen) ganz rechts bei 1.0 und die grünen ganz links bei 0.0.
- Der gestrichelte Strich ist der **optimale Threshold**, ab dem Alarm geschlagen wird.

**ROC-Kurve (Unten):**
- Zeigt das Verhältnis von Trefferquote zu Fehlalarmquote.
- Eine Kurve nahe der oberen linken Ecke (AUC nahe 1.0) ist perfekt.
- Die Diagonale wäre reines Raten (AUC 0.5).

In [18]:
# 6. DEEP LEARNING (BERT)
# Wir trainieren ein DistilBERT-Modell für maximale Genauigkeit.

# Für BERT brauchen wir die Rohdaten (X_train['text']), da der Tokenizer im Modell steckt.
# Wir nutzen hier df_Cleaning['text'] (bereinigt um Emojis/Shorts) oder df_train['text'] (Rohdaten).
# BERT profitiert oft von leicht bereinigten, aber nicht lemmatisierten Daten.
bert_model = run_bert_training(df_Cleaning['text'], df_Cleaning['target'], df_val['text'], df_val['target'])

### START: BERT Pipeline ###
### System-Check ###
OS: Darwin 24.6.0
CPU Kerne: 8
RAM: 16.00 GB
✅ GPU gefunden: 1x [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2026-01-08 02:51:02.824588: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2026-01-08 02:51:02.825110: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-01-08 02:51:02.825123: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1767837062.825675  533478 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1767837062.825977  533478 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/5


...

              precision    recall  f1-score   support

           0       0.88      0.66      0.75       869
           1       0.66      0.88      0.76       654

    accuracy                           0.75      1523
   macro avg       0.77      0.77      0.75      1523
weighted avg       0.79      0.75      0.75      1523




📊 NUMERISCHE MATRIX & METRIKEN:


,Vorhergesagt: Normal,Vorhergesagt: Katastrophe
Tatsächlich: Normal,570,299
Tatsächlich: Katastrophe,76,578



📊 ROC-KURVEN STÜTZPUNKTE (Auszug):


,False Positive Rate,True Positive Rate,Threshold
0,0.0000,0.0000,inf
45,0.0173,0.5520,0.9950
90,0.0552,0.6728,0.9876
135,0.0955,0.7401,0.9653
180,0.1507,0.8043,0.8970
225,0.2474,0.8440,0.7308
270,0.3475,0.8869,0.4913
315,0.5040,0.9251,0.2190
360,0.7227,0.9633,0.0744
406,1.0000,1.0000,0.0050


### 📊 Interpretation: BERT Trainingsverlauf & Ergebnisse
**Trainingsverlauf:**
- Zeigt Loss (Fehler) und Accuracy über die Epochen.
- **Wichtig**: Wenn der "Validation Loss" steigt, während der "Training Loss" sinkt, beginnt das Modell auswendig zu lernen (**Overfitting**). Early Stopping verhindert das.

**BERT Performance:**
- BERT versteht den Kontext von Wörtern (z.B. Sarkasmus) oft besser als klassische Modelle.
- Vergleichen Sie die Konfusionsmatrix hier mit der des Ensembles. Hat BERT weniger "Verpasste Gefahren" (False Negatives)?

In [19]:
# 7. FEATURE IMPORTANCE
# Wir schauen uns an, welche Wörter für das Ensemble-Modell am wichtigsten waren.
# Wir extrahieren den LogReg-Teil aus dem Ensemble für die Visualisierung
try:
    # Zugriff auf den VotingClassifier -> Estimators -> LogisticRegression
    logreg = best_ensemble.named_steps['clf'].estimators_[0]
    # Zugriff auf den TfidfVectorizer im Preprocessor
    tfidf = best_ensemble.named_steps['prep'].transformers_[0][1]
    feature_names = tfidf.get_feature_names_out()
    
    plot_feature_importance(logreg, feature_names, top_n=20)
except Exception as e:
    print(f"Feature Importance konnte nicht erstellt werden: {e}")


📊 TOP 20 FEATURE-WERTE (Numerische Daten):


,Feature,Importance
44398,train,5.1900
20933,hundred,4.7207
7820,collapse,4.3914
26729,massacre,4.2561
45610,typhoon,4.2483
5863,california,3.9182
24580,let,-3.9083
57632,Eng_Feature_57633,3.8539
53145,Eng_Feature_53146,3.8376
32858,police,3.7427


### 📊 Interpretation: Feature Importance
Dieser Plot zeigt, welche Wörter das Modell am stärksten beeinflussen.
- **Blaue Balken (Positiv)**: Wörter, die stark für "Katastrophe" sprechen (z.B. "hiroshima", "wildfire").
- **Rote Balken (Negativ)**: Wörter, die stark gegen eine Katastrophe sprechen.
- **Plausibilitäts-Check**: Sind hier Wörter dabei, die keinen Sinn ergeben (z.B. "the", "and")? Falls ja, muss die Stoppwort-Liste erweitert werden.

# 8. FAZIT & PHILOSOPHIE DES PROJEKTS
### Zusammenfassung der Ergebnisse

Wir haben eine umfassende Analyse von Katastrophen-Tweets durchgeführt, beginnend mit einer detaillierten explorativen Datenanalyse bis hin zum Training modernster Deep-Learning-Modelle.

**Wichtige Erkenntnisse:**
1.  **Datenqualität**: Die Bereinigung (Entfernen von URLs, Emojis-Handling) war entscheidend, um das Rauschen zu reduzieren.
2.  **Modell-Vergleich**: Der Benchmark zeigte, dass einfache Modelle wie Logistic Regression extrem schnell und solide sind, aber komplexe Modelle (Ensembles, BERT) bei der Feinheit (F1-Score) oft die Nase vorn haben.
3.  **Deep Learning**: Das BERT-Modell konnte durch sein Kontextverständnis oft Nuancen erkennen, die klassischen Modellen entgingen (z.B. Sarkasmus oder metaphorische Verwendung von "fire").

### Philosophie & Wiederverwendbarkeit für die Zukunft

Dieses Projekt wurde bewusst so strukturiert, dass es als **Blaupause für zukünftige NLP-Projekte** dient. Die Kernidee ist die Trennung von Ausführung (dieses Notebook) und Logik (die `nlp_utils.py` Bibliothek).

**Vorteile dieses Ansatzes:**
- **Vollständigkeit & Korrektheit**: Alle wichtigen Funktionen – von der EDA über den Modell-Benchmark bis zur Visualisierung – sind zentral in `nlp_utils.py` gesammelt. Dadurch wird sichergestellt, dass bei neuen Projekten keine wichtigen Analyseschritte vergessen werden. Es ist eine Checkliste in Codeform.
- **Wiederverwendbarkeit**: Für ein neues Projekt (z.B. Spam-Erkennung) muss nur die globale Konfiguration in `nlp_utils.py` (Pfade, Spaltennamen) angepasst werden. Die `run_comprehensive_benchmark` Funktion kann sofort auf die neuen Daten angewendet werden, um schnell das beste Modell zu finden.
- **Wissensspeicher**: Die Bibliothek dient als wachsender Wissensspeicher. Jede neue Funktion, jedes bessere Modell und jede neue Visualisierung, die wir entwickeln, wird dort hinzugefügt. So geht kein Code verloren und zukünftige Projekte profitieren automatisch vom gesammelten Wissen.
- **Sauberkeit & Fokus**: Das Notebook bleibt sauber, lesbar und fokussiert sich auf die Interpretation der Ergebnisse, während die komplexe Implementierung in der `.py`-Datei gekapselt ist.

Dieser Aufbau garantiert, dass wir bei jedem neuen Projekt auf einem soliden, vollständigen und professionellen Fundament aufbauen.


# Alle abildungen werden nicht gezeigt da diese mit Plotli geschrieben sin